# ONC Exploration

In [6]:
import requests
import pandas as pd
import json
from getpass import getpass
token = getpass(' Go to https://data.oceannetworks.ca/Profile and retrieve your Webservices API Token')

 Go to https://data.oceannetworks.ca/Profile and retrieve your Webservices API Token ········


In [7]:
# define the basic request query code snippet

def request_query(url,df=True):
    parameters = {'method':'get',
            'token':token}

    response = requests.get(url,params=parameters)
    #Handling the response fromt the query
    if (response.ok):
        if df:
            # write the info from query in a pandas dataframe
            df = pd.DataFrame(response.json()) # convert the json response to an object and load into a dataframe object
            return(df)
        else:
            return(response)
    else:
        if(response.status_code == 400):
            error = json.loads(str(response.content,'utf-8'))
            print(error) # json response contains a list of errors, with an errorMessage and parameter
        else:
            print ('Error {} - {}'.format(response.status_code,response.reason))

In [8]:
# Use the location services and the instrument category "Hydrophone" to list all locations
# where these type of instruments where installed over time

url ='https://data.oceannetworks.ca/api/locations?deviceCategoryCode=HYDROPHONE'

df = request_query(url)

In [22]:
df.loc[2]

deployments                                                        1
locationName                                            Hydrophone A
depth                                                          641.0
bbox               {'maxDepth': 641.0, 'maxLat': 48.345174, 'maxL...
description         Hydrophone at position A in a box-type array ...
hasDeviceData                                                   true
lon                                                      -126.157316
locationCode                                                BACNH.H1
hasPropertyData                                                false
lat                                                        48.345174
dataSearchURL      https://data.oceannetworks.ca/DataSearch?locat...
Name: 2, dtype: object

In [23]:
locCode = df.loc[2]['locationCode']
url  ='https://data.oceannetworks.ca/api/deployments?deviceCategoryCode=HYDROPHONE&locationCode='+locCode

In [25]:
df_depl = request_query(url)

In [26]:
df_depl

,begin,citation,depth,deviceCategoryCode,deviceCode,end,hasDeviceData,heading,lat,locationCode,lon,pitch,roll
0,2021-02-10T21:11:00.000Z,Ocean Networks Canada Society. 2021. Barkley C...,641.0,HYDROPHONE,ICLISTENHF6095,None,True,106.0,48.345174,BACNH.H1,-126.157316,None,None


In [10]:
# Query the deployment services with instrument category "Hydrophone" and location identifier "locationCode" from
# the previous results

df2 = pd.DataFrame(columns=['locationCode','begin','end','deviceCode','lat','lon','depth','hasDeviceData'])
for locCode in df['locationCode']:
    url  ='https://data.oceannetworks.ca/api/deployments?deviceCategoryCode=HYDROPHONE&locationCode='+locCode
    df_depl = request_query(url)
    # cutting out some columns that are not useful for the moment, you can leave the next line out and it will be
    # included into the full dataframe

    df_depl = df_depl[['locationCode','begin','end','deviceCode','lat','lon','depth','hasDeviceData']]
    df2 = pd.concat([df2,df_depl])

In [16]:
df2[df2['locationCode']=='BACND']

,locationCode,begin,end,deviceCode,lat,lon,depth,hasDeviceData
0,BACND,2018-06-22T03:05:27.000Z,2019-09-17T01:11:56.000Z,ICLISTENHF1252,48.345900,-126.157938,647.0,True
1,BACND,2019-09-17T01:11:56.000Z,2020-06-01T21:37:15.000Z,ICLISTENHF1252,48.345773,-126.157685,643.0,True
2,BACND,2022-05-21T00:57:21.000Z,2023-07-16T21:00:33.000Z,ICLISTENHF1252,48.345501,-126.158262,649.0,True


In [30]:
url =('https://data.oceannetworks.ca/api/archivefile-location?locationCode=NC89.H1&deviceCategoryCode=HYDROPHONE&'
      'dateFrom=2016-06-25&dateTo=2016-07-04&fileExtension=wav&dataProductCode=AD&returnOptions=all')

response = request_query(url)

ValueError: All arrays must be of the same length

In [ ]:
response